# 🛰️ Clustering Earth Observation Data: K-Means vs BIRCH

## 1. Introduction

In this practical, we will explore two popular **unsupervised learning algorithms** — **K-Means** and **BIRCH** — and apply them to the **EuroSAT RGB dataset**. The goal is to identify natural groupings (clusters) in satellite imagery without using labels.

### Why Clustering Earth Observation Data?
Satellite data is abundant and often unlabeled. Clustering helps discover structure in this data, e.g., separating vegetation, urban areas, or water bodies — useful in land-use mapping and environmental monitoring.

### What We'll Learn
- Load and preprocess EuroSAT RGB data
- Reduce data dimensionality with **PCA**
- Apply **K-Means** and **BIRCH** clustering
- Compare performance and visualize clusters

### Dataset Overview
The **EuroSAT RGB** dataset contains 27,000 Sentinel-2 images (64×64×3) representing 10 land-cover classes:
- Annual Crop  - Forest - Herbaceous Vegetation - Highway - Industrial  
- Pasture - Permanent Crop - Residential - River - Sea/Lake

[EuroSAT dataset](https://github.com/phelber/EuroSAT)

![](https://raw.githubusercontent.com/phelber/EuroSAT/master/eurosat_overview_small.jpg)

In [ ]:
import requests
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, Birch
import urllib.request, zipfile, os, time, warnings
from sklearn.metrics import silhouette_score, calinski_harabasz_score

warnings.filterwarnings('ignore')

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 2. Data Acquisition and Loading

We'll download the EuroSAT RGB dataset from **Zenodo** and extract a manageable subset of images for this lesson (e.g., 300 per class). Each image is 64×64 pixels with three color channels (RGB).

In [ ]:
def download_eurosat():
    """
    Download the official EuroSAT RGB dataset from DFKI (SSL verification disabled).
    """
    url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"
    zip_path = "EuroSAT.zip"
    target_dir = "EuroSAT/2750"

    if not os.path.exists(target_dir):
        print("⬇️ Downloading EuroSAT RGB dataset from DFKI...")
        response = requests.get(url, verify=False, stream=True)
        total = int(response.headers.get('content-length', 0))
        with open(zip_path, 'wb') as f:
            downloaded = 0
            for data in response.iter_content(chunk_size=8192):
                f.write(data)
                downloaded += len(data)
                done = int(50 * downloaded / total)
                print(f"\r[{'=' * done}{' ' * (50 - done)}] {downloaded/1e6:.1f}/{total/1e6:.1f} MB", end='')
        print("\n📦 Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall("EuroSAT")
        os.remove(zip_path)
        print("✅ Dataset downloaded and extracted successfully!")
    else:
        print("✅ Dataset already available.")

def load_images(data_dir="EuroSAT/2750", max_per_class=300):
    """
    Load RGB images from the EuroSAT dataset.
    Each image is 64x64x3.
    """
    images, labels, classes = [], [], []
    for class_dir in sorted(Path(data_dir).iterdir()):
        if class_dir.is_dir():
            cls = class_dir.name
            classes.append(cls)
            files = list(class_dir.glob("*.jpg"))[:max_per_class]
            for f in files:
                img = np.array(Image.open(f))
                images.append(img)
                labels.append(cls)
    return np.array(images), np.array(labels), classes


# Run download + load
download_eurosat()
images, true_labels, class_names = load_images(max_per_class=300)
print(f"✅ Loaded {len(images)} images from {len(class_names)} classes.")


## 3. Exploring the Dataset

Let's take a quick look at the data by plotting one sample from each class. Visual inspection helps confirm that the dataset loaded correctly and provides intuition for clustering.

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15,6))
fig.suptitle('Example Images from EuroSAT (RGB)', fontsize=14, fontweight='bold')
for i, cls in enumerate(class_names):
    idx = np.where(true_labels == cls)[0][0]
    axes[i//5, i%5].imshow(images[idx])
    axes[i//5, i%5].set_title(cls.replace('_',' '))
    axes[i//5, i%5].axis('off')
plt.tight_layout()
plt.show()

## 4. Dimensionality Reduction with PCA

Each image has 64×64×3 = **12,288 features**. Clustering directly in this high-dimensional space is computationally expensive and prone to the *curse of dimensionality*.

We use **Principal Component Analysis (PCA)** to:
- Reduce redundancy among correlated features (RGB pixels)
- Keep the components explaining most variance
- Make clustering faster and more effective

We'll project down to **50 components**, typically capturing ~90% of the data variance.

In [ ]:
X = images.reshape(len(images), -1) / 255.0
pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X)
print(f"Reduced from {X.shape[1]} to {X_pca.shape[1]} features.")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.2%}")

Let's check how much variance each PCA component captures. This helps justify our choice of 50 components.

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by PCA Components')
plt.grid(True, alpha=0.4)
plt.show()

## 5. K-Means Clustering


**What it is**:

K-Means is a partition-based clustering algorithm. It tries to group your data into K clusters so that items in the same cluster are similar, and items in different clusters are as different as possible.


**How it works (simplified):**

Pick K random points as initial cluster centers (called centroids).

Assign each data point to the nearest centroid.

Update each centroid to be the mean of all points assigned to it.

Repeat steps 2–3 until centroids stop moving or a maximum number of iterations is reached.


**Key points:**

You must decide the number of clusters (K) beforehand.

Works best when clusters are roughly spherical and similar in size.

Sensitive to initial centroid positions — running it multiple times with different seeds usually gives better results.

Fast and widely used, but it might struggle with irregular cluster shapes.

**Why use it here:**

K-Means can help identify patterns in satellite images, like grouping areas with similar land cover (forest, river, urban, etc.), even if we don’t tell the algorithm what the classes are.

**Algorithm Steps:**
1. Initialize K centroids randomly
2. Assign each point to its nearest centroid
3. Update centroids as the mean of assigned points
4. Repeat until convergence

We’ll set *K = 10* (same as the number of known classes) and evaluate clustering quality.

In [ ]:
n_clusters = len(class_names)
start = time.time()
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
labels_km = kmeans.fit_predict(X_pca)
time_km = time.time() - start

sil_km = silhouette_score(X_pca, labels_km)
ch_km = calinski_harabasz_score(X_pca, labels_km)

print(f"K-Means completed in {time_km:.2f}s")
print(f"Silhouette Score: {sil_km:.3f}, Calinski-Harabasz Score: {ch_km:.1f}")

### Interpreting Metrics
- **Silhouette Score** ∈ [-1, 1]: Higher values indicate better separation.
- **Calinski–Harabasz Index**: Larger values → well-separated, compact clusters.

These metrics don't rely on labels, so they’re ideal for unsupervised evaluation.

## 6. BIRCH Clustering

**What it is:**

BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies) is a hierarchical clustering algorithm designed for large datasets.

**How it works (simplified):**

Builds a CF Tree (Clustering Feature Tree) that summarizes all points in the dataset.

Each node in the tree keeps track of cluster statistics (like the number of points, their sum, and sum of squares).

New points are incrementally added to the tree without storing all data in memory.

After the tree is built, clusters can be extracted from the summarized nodes.

**Key points:**

Very memory-efficient; can handle large datasets that don’t fit entirely in memory.

Can automatically discover clusters if n_clusters=None.

Sensitive to the order of the data, so results might slightly change depending on the input order.

Works well with large, high-dimensional data like images, especially after PCA.

**Why use it here:**

BIRCH is ideal for satellite images because it can quickly summarize and cluster thousands of images without consuming too much memory. It gives a different perspective from K-Means, allowing comparison of clustering strategies.

**Advantages:**

- Handles massive data efficiently
- Works incrementally
- Can automatically discover number of clusters

We'll again set `n_clusters=10` for comparability.

In [ ]:
start = time.time()
birch = Birch(n_clusters=n_clusters, threshold=0.5)
labels_birch = birch.fit_predict(X_pca)
time_br = time.time() - start

sil_br = silhouette_score(X_pca, labels_birch)
ch_br = calinski_harabasz_score(X_pca, labels_birch)

print(f"BIRCH completed in {time_br:.2f}s")
print(f"Silhouette Score: {sil_br:.3f}, Calinski-Harabasz Score: {ch_br:.1f}")

## 7. Comparing Algorithm Performance

Let's summarize the metrics and compare runtime and clustering quality between K-Means and BIRCH.

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Algorithm': ['K-Means', 'BIRCH'],
    'Time (s)': [time_km, time_br],
    'Silhouette Score': [sil_km, sil_br],
    'Calinski-Harabasz': [ch_km, ch_br]
})
display(df)

df.plot(x='Algorithm', y=['Time (s)', 'Silhouette Score'], kind='bar', subplots=True, layout=(1,2), figsize=(12,4), legend=False)
plt.suptitle('Performance Comparison: K-Means vs BIRCH', fontweight='bold')
plt.show()

## 8. Visualizing Clusters in 2D

To better interpret the clusters, we’ll use another PCA projection to reduce our 50D data to 2D for plotting. Although the 2D projection can’t fully represent high-dimensional clusters, it offers a helpful visual intuition.

In [ ]:
pca_2d = PCA(n_components=2, random_state=42)
X_2d = pca_2d.fit_transform(X_pca)

fig, ax = plt.subplots(1,2, figsize=(15,6))
ax[0].scatter(X_2d[:,0], X_2d[:,1], c=labels_km, cmap='tab10', s=10)
ax[0].set_title('K-Means Clusters (2D PCA view)')
ax[1].scatter(X_2d[:,0], X_2d[:,1], c=labels_birch, cmap='tab10', s=10)
ax[1].set_title('BIRCH Clusters (2D PCA view)')
plt.show()

## 9. Inspecting Cluster Contents

We’ll visually inspect which kinds of images appear in each cluster. If clusters are semantically meaningful, similar landscapes (e.g., forests, water bodies) should appear together.

In [ ]:
def show_cluster_samples(images, labels, algo, n_samples=5):
    clusters = np.unique(labels)
    fig, axes = plt.subplots(len(clusters), n_samples, figsize=(12,2.5*len(clusters)))
    fig.suptitle(f'{algo}: Example Images per Cluster', fontsize=14, fontweight='bold')
    for i, c in enumerate(clusters):
        idxs = np.where(labels==c)[0]
        idxs = np.random.choice(idxs, min(n_samples, len(idxs)), replace=False)
        for j, idx in enumerate(idxs):
            axes[i,j].imshow(images[idx])
            axes[i,j].axis('off')
    plt.tight_layout()
    plt.show()

show_cluster_samples(images, labels_km, 'K-Means')
show_cluster_samples(images, labels_birch, 'BIRCH')

## 10. Conclusions

**Summary of Findings:**
- PCA reduced image dimensionality effectively (~90% variance retained).
- **K-Means** achieved slightly higher silhouette and CH scores — better compactness.
- **BIRCH** was faster and more scalable but sometimes less stable.

**Takeaways:**
- PCA is essential for high-dimensional EO data.
- K-Means suits well-separated, compact clusters.
- BIRCH scales better for large or streaming data.

**Next Steps:**
- Try tuning PCA components (20–100) and BIRCH threshold.
- Experiment with subsets (urban vs. rural scenes).
- Explore t-SNE or UMAP for nonlinear dimensionality reduction.